## CAPM

1. Tính toán Hệ số Beta và Rủi ro Phi hệ thống: sử dụng phương pháp Hồi quy tuyến tính (OLS Regression) để tìm mối quan hệ giữa lợi suất của từng cổ phiếu (biến phụ thuộc y) và lợi suất của thị trường VN-INDEX (biến độc lập x). Từ đó, trích xuất ra hệ số beta (đo lường rủi ro hệ thống) và phương sai của phần dư (đo lường rủi ro phi hệ thống).

2. Dựa vào hệ số Beta tính được, mã nguồn tự động xếp loại các ngân hàng vào 3 nhóm:
- Rủi ro thấp (beta<1)
- Rủi ro trung bình (-1<beta<1.2)
- Rủi co cao (beta>1.2)

3. Xuất báo cáo kết quả: Mã nguồn lưu trữ kết quả vào 2 file CSV
- capm_beta_results.csv: Bảng đầy đủ các chỉ số thống kê
- beta_summary_table.csv: Bảng rút gọn chỉ gồm Ticker và Beta.


In [ ]:
# Chạy mô hình CAPM (Ch 3.4)
# PHIÊN BẢN CẬP NHẬT: Đã chỉnh sửa đường dẫn thư mục data_ohlcv

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import os
import sys

# --- 1. CẤU HÌNH ĐƯỜNG DẪN ---
DATA_FOLDER = 'data_ohlcv'
INPUT_FILE = 'data_final.csv'
INPUT_PATH = os.path.join(DATA_FOLDER, INPUT_FILE)

# Tên file kết quả đầu ra (sẽ lưu vào cùng thư mục data)
OUTPUT_FULL = os.path.join(DATA_FOLDER, 'capm_beta_results.csv')
OUTPUT_SUMMARY = os.path.join(DATA_FOLDER, 'beta_summary_table.csv')

def calculate_capm_beta(df, bank_tickers):
    """
    Tính hệ số Beta và Rủi ro Phi hệ thống (Var(epsilon_t)) cho các mã 
    cổ phiếu ngân hàng bằng mô hình CAPM (OLS).
    """
    
    results = []
    
    print("=" * 70)
    print("BẮT ĐẦU TÍNH TOÁN HỆ SỐ BETA & RỦI RO PHI HỆ THỐNG")
    print("=" * 70)
    
    # Lọc ra CHỈ các cổ phiếu ngân hàng
    df_stocks = df[df['ticker'].isin(bank_tickers)].copy()
    
    for ticker in bank_tickers:
        print(f"\n📊 Đang xử lý: {ticker}")
        
        # 1. Lọc dữ liệu cho cổ phiếu hiện tại
        # Cần đảm bảo cột VNI_log_return (đã được merge từ bước trước) tồn tại
        if 'VNI_log_return' not in df_stocks.columns:
             print("❌ LỖI: Không tìm thấy cột 'VNI_log_return'. Vui lòng chạy lại bước Feature Engineering.")
             sys.exit()

        stock_data = df_stocks[df_stocks['ticker'] == ticker][['log_return', 'VNI_log_return']].copy()
        
        # 2. Loại bỏ các giá trị NaN
        stock_data = stock_data.dropna(subset=['log_return', 'VNI_log_return'])
        
        # 3. Kiểm tra nếu có đủ dữ liệu
        if len(stock_data) < 30: # Cần ít nhất 30 quan sát
            print(f"  ⚠️  Không đủ dữ liệu ({len(stock_data)} quan sát) sau khi loại bỏ NaN")
            results.append({
                'Ticker': ticker, 'Beta': np.nan, 'Alpha': np.nan,
                'R_squared': np.nan, 
                'Unsystematic_Risk_Var': np.nan,
                'P_value': np.nan,
                'Std_Error': np.nan, 'N_observations': len(stock_data)
            })
            continue
            
        # 4. Chuẩn bị dữ liệu cho hồi quy
        y = stock_data['log_return'].values
        X = stock_data['VNI_log_return'].values 
        
        # 5. Chạy OLS
        X_with_const = sm.add_constant(X)
        model = sm.OLS(y, X_with_const).fit()
        
        # Lấy kết quả
        alpha = model.params[0]
        beta = model.params[1]
        r_squared = model.rsquared
        p_value = model.pvalues[1]
        std_error = model.bse[1]
        
        # Tính Rủi ro Phi hệ thống (Var(epsilon))
        unsystematic_risk_var = model.mse_resid
        
        # 6. Lưu kết quả
        results.append({
            'Ticker': ticker,
            'Beta': beta,
            'Alpha': alpha,
            'R_squared': r_squared,
            'Unsystematic_Risk_Var': unsystematic_risk_var,
            'P_value': p_value,
            'Std_Error': std_error,
            'N_observations': len(stock_data)
        })
        
        # In kết quả chi tiết
        print(f"  ✓ Beta: {beta:.4f}")
        print(f"  ✓ R²: {r_squared:.4f}")
        print(f"  ✓ Rủi ro phi hệ thống (Var(ε)): {unsystematic_risk_var:.8f}")
        print(f"  ✓ Số quan sát: {len(stock_data)}")
        
    # Tạo DataFrame kết quả và sắp xếp
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df = results_df.sort_values('Beta', ascending=False).reset_index(drop=True)
    
    return results_df

def print_summary_statistics(results_df):
    """
    In thống kê tóm tắt về Beta
    """
    print("\n" + "=" * 70)
    print("THỐNG KÊ TÓM TẮT VỀ HỆ SỐ BETA")
    print("=" * 70)
    
    if results_df.empty or 'Beta' not in results_df.columns:
        print("❌ Không có dữ liệu kết quả.")
        return

    valid_betas = results_df['Beta'].dropna()
    
    print(f"\n📈 Số mã có Beta hợp lệ: {len(valid_betas)}/{len(results_df)}")
    
    if valid_betas.empty:
        print("   ⚠️ Không có dữ liệu Beta hợp lệ.")
        return

    print(f"📊 Beta trung bình: {valid_betas.mean():.4f}")
    print(f"📊 Beta trung vị: {valid_betas.median():.4f}")
    
    max_idx = results_df['Beta'].idxmax()
    min_idx = results_df['Beta'].idxmin()
    print(f"📊 Beta cao nhất: {valid_betas.max():.4f} ({results_df.loc[max_idx, 'Ticker']})")
    print(f"📊 Beta thấp nhất: {valid_betas.min():.4f} ({results_df.loc[min_idx, 'Ticker']})")
    print(f"📊 Độ lệch chuẩn: {valid_betas.std():.4f}")
    
    print("\n📋 PHÂN LOẠI THEO MỨC ĐỘ RỦI RO:")
    defensive = results_df[results_df['Beta'] < 1.0]
    neutral = results_df[(results_df['Beta'] >= 1.0) & (results_df['Beta'] <= 1.2)]
    aggressive = results_df[results_df['Beta'] > 1.2]
    
    print(f"  🛡️  Defensive (Beta < 1.0): {len(defensive)} mã")
    print(f"  ⚖️  Neutral (1.0 ≤ Beta ≤ 1.2): {len(neutral)} mã")
    print(f"  🚀 Aggressive (Beta > 1.2): {len(aggressive)} mã")


# ============================================================================
# CHƯƠNG TRÌNH CHÍNH
# ============================================================================

if __name__ == "__main__":
    
    print(f"📂 Đang đọc file từ: {INPUT_PATH}...")
    
    # 1. Kiểm tra file tồn tại
    if not os.path.exists(INPUT_PATH):
        print(f"❌ LỖI: Không tìm thấy file '{INPUT_PATH}'.")
        print("   Vui lòng kiểm tra lại tên thư mục hoặc chạy lại bước Feature Engineering.")
        sys.exit()

    # 2. Đọc dữ liệu
    try:
        df = pd.read_csv(INPUT_PATH)
        print(f"✓ Đã đọc {len(df)} dòng dữ liệu")
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        sys.exit()
        
    # 3. Chuyển đổi cột 'date'
    try:
        df['date'] = pd.to_datetime(df['date'])
        print("✓ Đã chuyển đổi cột 'date' sang định dạng datetime.")
    except Exception as e:
        print(f"LỖI: Không thể chuyển đổi cột 'date'. Lỗi: {e}")
        sys.exit()
    
    # Danh sách ngân hàng cần tính
    bank_tickers = ['ACB', 'BID', 'CTG', 'HDB', 'LPB', 'MBB', 
                    'EIB', 'STB', 'TCB', 'TPB', 'VCB', 'VIB', 
                    'VPB', 'SHB'] # Đã bỏ các mã rác như MSB, NAB...
    
    # Kiểm tra xem mã có trong file không
    available_tickers = df['ticker'].unique()
    missing_tickers = [t for t in bank_tickers if t not in available_tickers]
    
    if missing_tickers:
        print(f"\n⚠️  Cảnh báo: Các mã sau không có trong dữ liệu (có thể đã bị lọc): {missing_tickers}")
        bank_tickers = [t for t in bank_tickers if t in available_tickers]
        
    print(f"\n📋 Sẽ tính Beta cho {len(bank_tickers)} mã ngân hàng")
    
    # 4. Tính Beta
    results_df = calculate_capm_beta(df, bank_tickers)
    
    # 5. In thống kê
    print_summary_statistics(results_df)
    
    # 6. Hiển thị và Lưu kết quả
    if not results_df.empty:
        print("\n" + "=" * 70)
        print("BẢNG KẾT QUẢ CHI TIẾT")
        print("=" * 70)
        
        cols_to_round = {
            'Beta': 4, 
            'Alpha': 6, 
            'R_squared': 4, 
            'Unsystematic_Risk_Var': 8,
            'P_value': 4, 
            'Std_Error': 6
        }
        
        results_df_rounded = results_df.round(cols_to_round)
        print(results_df_rounded.to_string(index=False))
        
        # Lưu file kết quả đầy đủ
        results_df.to_csv(OUTPUT_FULL, index=False, encoding='utf-8-sig')
        print(f"\n✅ Đã lưu kết quả (đầy đủ) vào file: {OUTPUT_FULL}")
        
        # Lưu file tóm tắt
        simple_table = results_df[['Ticker', 'Beta']].copy()
        simple_table.to_csv(OUTPUT_SUMMARY, index=False, encoding='utf-8-sig')
        print(f"✅ Đã lưu bảng tóm tắt (Ticker, Beta) vào file: {OUTPUT_SUMMARY}")
    
    print("\n" + "=" * 70)
    print("HOÀN THÀNH!")
    print("=" * 70)# Chạy mô hình CAPM (Ch 3.4)
# PHIÊN BẢN CẬP NHẬT: Đã chỉnh sửa đường dẫn thư mục data_ohlcv

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import os
import sys

# --- 1. CẤU HÌNH ĐƯỜNG DẪN ---
DATA_FOLDER = 'data_ohlcv'
INPUT_FILE = 'data_final.csv'
INPUT_PATH = os.path.join(DATA_FOLDER, INPUT_FILE)

# Tên file kết quả đầu ra (sẽ lưu vào cùng thư mục data)
OUTPUT_FULL = os.path.join(DATA_FOLDER, 'capm_beta_results.csv')
OUTPUT_SUMMARY = os.path.join(DATA_FOLDER, 'beta_summary_table.csv')

def calculate_capm_beta(df, bank_tickers):
    """
    Tính hệ số Beta và Rủi ro Phi hệ thống (Var(epsilon_t)) cho các mã 
    cổ phiếu ngân hàng bằng mô hình CAPM (OLS).
    """
    
    results = []
    
    print("=" * 70)
    print("BẮT ĐẦU TÍNH TOÁN HỆ SỐ BETA & RỦI RO PHI HỆ THỐNG")
    print("=" * 70)
    
    # Lọc ra CHỈ các cổ phiếu ngân hàng
    df_stocks = df[df['ticker'].isin(bank_tickers)].copy()
    
    for ticker in bank_tickers:
        print(f"\n📊 Đang xử lý: {ticker}")
        
        # 1. Lọc dữ liệu cho cổ phiếu hiện tại
        # Cần đảm bảo cột VNI_log_return (đã được merge từ bước trước) tồn tại
        if 'VNI_log_return' not in df_stocks.columns:
             print("❌ LỖI: Không tìm thấy cột 'VNI_log_return'. Vui lòng chạy lại bước Feature Engineering.")
             sys.exit()

        stock_data = df_stocks[df_stocks['ticker'] == ticker][['log_return', 'VNI_log_return']].copy()
        
        # 2. Loại bỏ các giá trị NaN
        stock_data = stock_data.dropna(subset=['log_return', 'VNI_log_return'])
        
        # 3. Kiểm tra nếu có đủ dữ liệu
        if len(stock_data) < 30: # Cần ít nhất 30 quan sát
            print(f"  ⚠️  Không đủ dữ liệu ({len(stock_data)} quan sát) sau khi loại bỏ NaN")
            results.append({
                'Ticker': ticker, 'Beta': np.nan, 'Alpha': np.nan,
                'R_squared': np.nan, 
                'Unsystematic_Risk_Var': np.nan,
                'P_value': np.nan,
                'Std_Error': np.nan, 'N_observations': len(stock_data)
            })
            continue
            
        # 4. Chuẩn bị dữ liệu cho hồi quy
        y = stock_data['log_return'].values
        X = stock_data['VNI_log_return'].values 
        
        # 5. Chạy OLS
        X_with_const = sm.add_constant(X)
        model = sm.OLS(y, X_with_const).fit()
        
        # Lấy kết quả
        alpha = model.params[0]
        beta = model.params[1]
        r_squared = model.rsquared
        p_value = model.pvalues[1]
        std_error = model.bse[1]
        
        # Tính Rủi ro Phi hệ thống (Var(epsilon))
        unsystematic_risk_var = model.mse_resid
        
        # 6. Lưu kết quả
        results.append({
            'Ticker': ticker,
            'Beta': beta,
            'Alpha': alpha,
            'R_squared': r_squared,
            'Unsystematic_Risk_Var': unsystematic_risk_var,
            'P_value': p_value,
            'Std_Error': std_error,
            'N_observations': len(stock_data)
        })
        
        # In kết quả chi tiết
        print(f"  ✓ Beta: {beta:.4f}")
        print(f"  ✓ R²: {r_squared:.4f}")
        print(f"  ✓ Rủi ro phi hệ thống (Var(ε)): {unsystematic_risk_var:.8f}")
        print(f"  ✓ Số quan sát: {len(stock_data)}")
        
    # Tạo DataFrame kết quả và sắp xếp
    results_df = pd.DataFrame(results)
    if not results_df.empty:
        results_df = results_df.sort_values('Beta', ascending=False).reset_index(drop=True)
    
    return results_df

def print_summary_statistics(results_df):
    """
    In thống kê tóm tắt về Beta
    """
    print("\n" + "=" * 70)
    print("THỐNG KÊ TÓM TẮT VỀ HỆ SỐ BETA")
    print("=" * 70)
    
    if results_df.empty or 'Beta' not in results_df.columns:
        print("❌ Không có dữ liệu kết quả.")
        return

    valid_betas = results_df['Beta'].dropna()
    
    print(f"\n📈 Số mã có Beta hợp lệ: {len(valid_betas)}/{len(results_df)}")
    
    if valid_betas.empty:
        print("   ⚠️ Không có dữ liệu Beta hợp lệ.")
        return

    print(f"📊 Beta trung bình: {valid_betas.mean():.4f}")
    print(f"📊 Beta trung vị: {valid_betas.median():.4f}")
    
    max_idx = results_df['Beta'].idxmax()
    min_idx = results_df['Beta'].idxmin()
    print(f"📊 Beta cao nhất: {valid_betas.max():.4f} ({results_df.loc[max_idx, 'Ticker']})")
    print(f"📊 Beta thấp nhất: {valid_betas.min():.4f} ({results_df.loc[min_idx, 'Ticker']})")
    print(f"📊 Độ lệch chuẩn: {valid_betas.std():.4f}")
    
    print("\n📋 PHÂN LOẠI THEO MỨC ĐỘ RỦI RO:")
    defensive = results_df[results_df['Beta'] < 1.0]
    neutral = results_df[(results_df['Beta'] >= 1.0) & (results_df['Beta'] <= 1.2)]
    aggressive = results_df[results_df['Beta'] > 1.2]
    
    print(f"  🛡️  Defensive (Beta < 1.0): {len(defensive)} mã")
    print(f"  ⚖️  Neutral (1.0 ≤ Beta ≤ 1.2): {len(neutral)} mã")
    print(f"  🚀 Aggressive (Beta > 1.2): {len(aggressive)} mã")


# ============================================================================
# CHƯƠNG TRÌNH CHÍNH
# ============================================================================

if __name__ == "__main__":
    
    print(f"📂 Đang đọc file từ: {INPUT_PATH}...")
    
    # 1. Kiểm tra file tồn tại
    if not os.path.exists(INPUT_PATH):
        print(f"❌ LỖI: Không tìm thấy file '{INPUT_PATH}'.")
        print("   Vui lòng kiểm tra lại tên thư mục hoặc chạy lại bước Feature Engineering.")
        sys.exit()

    # 2. Đọc dữ liệu
    try:
        df = pd.read_csv(INPUT_PATH)
        print(f"✓ Đã đọc {len(df)} dòng dữ liệu")
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        sys.exit()
        
    # 3. Chuyển đổi cột 'date'
    try:
        df['date'] = pd.to_datetime(df['date'])
        print("✓ Đã chuyển đổi cột 'date' sang định dạng datetime.")
    except Exception as e:
        print(f"LỖI: Không thể chuyển đổi cột 'date'. Lỗi: {e}")
        sys.exit()
    
    # Danh sách ngân hàng cần tính
    bank_tickers = ['ACB', 'BID', 'CTG', 'HDB', 'LPB', 'MBB', 
                    'EIB', 'STB', 'TCB', 'TPB', 'VCB', 'VIB', 
                    'VPB', 'SHB'] # Đã bỏ các mã rác như MSB, NAB...
    
    # Kiểm tra xem mã có trong file không
    available_tickers = df['ticker'].unique()
    missing_tickers = [t for t in bank_tickers if t not in available_tickers]
    
    if missing_tickers:
        print(f"\n⚠️  Cảnh báo: Các mã sau không có trong dữ liệu (có thể đã bị lọc): {missing_tickers}")
        bank_tickers = [t for t in bank_tickers if t in available_tickers]
        
    print(f"\n📋 Sẽ tính Beta cho {len(bank_tickers)} mã ngân hàng")
    
    # 4. Tính Beta
    results_df = calculate_capm_beta(df, bank_tickers)
    
    # 5. In thống kê
    print_summary_statistics(results_df)
    
    # 6. Hiển thị và Lưu kết quả
    if not results_df.empty:
        print("\n" + "=" * 70)
        print("BẢNG KẾT QUẢ CHI TIẾT")
        print("=" * 70)
        
        cols_to_round = {
            'Beta': 4, 
            'Alpha': 6, 
            'R_squared': 4, 
            'Unsystematic_Risk_Var': 8,
            'P_value': 4, 
            'Std_Error': 6
        }
        
        results_df_rounded = results_df.round(cols_to_round)
        print(results_df_rounded.to_string(index=False))
        
        # Lưu file kết quả đầy đủ
        results_df.to_csv(OUTPUT_FULL, index=False, encoding='utf-8-sig')
        print(f"\n✅ Đã lưu kết quả (đầy đủ) vào file: {OUTPUT_FULL}")
        
        # Lưu file tóm tắt
        simple_table = results_df[['Ticker', 'Beta']].copy()
        simple_table.to_csv(OUTPUT_SUMMARY, index=False, encoding='utf-8-sig')
        print(f"✅ Đã lưu bảng tóm tắt (Ticker, Beta) vào file: {OUTPUT_SUMMARY}")
    
    print("\n" + "=" * 70)
    print("HOÀN THÀNH!")
    print("=" * 70)